# Output Parser

- Extract the output from the model to the desired format.


`.get_format_instructions()`
This function provides the necessary instructions into your prompt to output a
structured format that can be parsed.

`.parse(llm_output: str)`
This function is responsible for parsing your LLM responses into a predefined
format.


principles = """

- The name must be easy to remember.
- The name must be easy to pronounce.
- Avoid returning full stops, \n, or any other characters.
- The maximum length of the name must be 10 characters.
  """


### Output Parser with parser.parse


In [8]:
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.prompts import SystemMessagePromptTemplate, ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic.v1 import BaseModel, Field
from typing import List

from dotenv import load_dotenv
load_dotenv()

# Define Model
model = ChatMistralAI()

# Define Pydantic Structure


class Character(BaseModel):
    name: str = Field(description="The name of the character")
    description: str = Field(
        description="The general description of this character")
    characteristic: str = Field(
        description="The adjective of this character, EG charisma")
    important: float = Field(
        description="How important this character in this story, score 1.0-10.0")
    type: str = Field(
        description="The type of the character, either good, bad, neutral")
    frequency: int = Field(
        description="The number of times will appear in the story")


class Characters(BaseModel):
    characters: List[Character] = Field(description="A list of character")


# Define Output Parser
parser = PydanticOutputParser(pydantic_object=Characters)

# Define Principle and Template
principles = """
- The name must be easy to remember.
- The name must be easy to pronounce.
- Avoid returning full stops, \n, or any other characters.
- The maximum length of the name must be 10 characters.
"""

template = """
Generate a list of {n} characters for the story {context}

The generation must follow this principles: {principles}
{format_instruction}
"""

# Define SystemMessagePromptTemplate and ChatPromptTemplate
system_prompt = SystemMessagePromptTemplate.from_template(template=template)
user_prompt = ChatPromptTemplate.from_messages([system_prompt])

# LCEL
chain = user_prompt | model

# Invoke
response = chain.invoke({
    "n": 5,
    "context": "In the twilight of a forgotten kingdom, a lone wanderer discovered an ancient key that whispered secrets of a lost world hidden beneath the earth.",
    "principles": principles,
    "format_instruction": parser.get_format_instructions(),
})

# Parser.parse to get back result
print(parser.parse(response.content))

characters=[Character(name='Eva', description='A kind-hearted blacksmith with a mysterious past', characteristic='Strong-willed', important=8.5, type='Good', frequency=8), Character(name='Kai', description='A cunning merchant who trades in rare antiquities', characteristic='Secretive', important=7.0, type='Neutral', frequency=6), Character(name='Din', description='A wise old sorcerer who is the guardian of the lost world', characteristic='Enigmatic', important=9.5, type='Good', frequency=5), Character(name='Nad', description='A ruthless bandit leader seeking the power of the lost world', characteristic='Fearless', important=8.0, type='Bad', frequency=7), Character(name='Lyn', description='A curious scholar who follows the lone wanderer', characteristic='Inquisitive', important=6.5, type='Neutral', frequency=9)]


### Output Parser with LCEL


In [9]:
# LCEL
chain = user_prompt | model | parser

# Invoke
response = chain.invoke({
    "n": 5,
    "context": "In the twilight of a forgotten kingdom, a lone wanderer discovered an ancient key that whispered secrets of a lost world hidden beneath the earth.",
    "principles": principles,
    "format_instruction": parser.get_format_instructions(),
})

print(response)

characters=[Character(name='Eva', description='A kind-hearted blacksmith with a mysterious past.', characteristic='Strong-willed', important=8.5, type='Good', frequency=8), Character(name='Cato', description='A cunning merchant who trades in rare antiquities.', characteristic='Secretive', important=7.0, type='Neutral', frequency=6), Character(name='Kai', description='A mischievous rogue with a heart of gold, often found in taverns.', characteristic='Witty', important=6.5, type='Good', frequency=7), Character(name='Lila', description='A wise old seer who lives in the outskirts of the kingdom.', characteristic='Mysterious', important=9.0, type='Good', frequency=5), Character(name='Drax', description='A ruthless warlord bent on conquering the forgotten kingdom.', characteristic='Cruel', important=10.0, type='Bad', frequency=9)]
